On commence par importer les bibliothèques nécessaires. Attention : NE PAS EFFACER !

In [ ]:
from Import_bibliotheques import *
# On initialise la carte Arduino avec le protocole telemetrix
from telemetrix import telemetrix

Conditions expérimentales :

In [ ]:
charge = 1    # mettre 1 pour la charge du condensateur, et 0 pour sa décharge
duree =  0.2    # Entrer la durée d'acquisition désirée en s
Analog_pin = 0    # numéro de la voie de mesure analogique
Digital_pin = 13    # muméro de la voie de commande digitale
intervalle = 5    # durée entre 2 mesures en ms (minimum = 1)

On se connecte à Arduino :

In [ ]:
uc = []
t = []
mesure = 0   # 0 pour interdire les mesures, 1 pour les autoriser
# Callback data indices
CB_PIN_MODE = 0
CB_PIN = 1
CB_VALUE = 2
CB_TIME = 3
carte = telemetrix.Telemetrix("/dev/ttyACM0")    # on définit la carte Arduino sous le nom de "carte"

def MonCallback(data) :    # on définit ici ce qui sera exécuté à chaque appel de mesure
    global mesure, t, uc, i, debut, duree
    raw_value = data[CB_VALUE]
    if mesure :
        if i == 0 :
            debut = data[CB_TIME]
        instant = data[CB_TIME] - debut
        if instant <= duree :
            t.append(instant)
            tension = raw_value * 5.0 / 1023
            uc.append(tension)
        i = 1

carte.set_pin_mode_analog_input(Analog_pin, callback = MonCallback)    # définition de la voie d'acquisition
carte.set_analog_scan_interval(intervalle)
carte.set_pin_mode_digital_output(Digital_pin)    # définition de la voie d'acquisition
carte.digital_write(Digital_pin, 1 - charge)    # on met l'alimentation sur 0 V ou 5V selon les conditions initiales
sleep(duree)

Interaction avec les voies définies précédemment :

In [ ]:
mesure = 1    # on va commencer les mesures
i = 0    # initialisation du compteur pour que le temps commence à 0
carte.digital_write(Digital_pin, charge)    # on met l'alimentation sur 0 V ou 5V selon les conditions initiales
sleep(duree)    # en s, attente de la (dé)charge complète du condensateur
carte.disable_analog_reporting(Analog_pin)    # on arrête les mesures
carte.shutdown()   # fermeture propre de la communication avec Arduino

# on transforme les listes en tableaux pour les traitements scientifiques :
t = array(t)
uc = array(uc)

Calcul des caractéristiques de la courbe de régression. Attention : NE PAS SUPPRIMER, et ne le modifier que si les paramètres par défaut ne donnent pas de bon résultat :

In [ ]:
def Modele(expression, x, y):
    
    modele = ExpressionModel(expression)    # si la variable indépendante n'est pas "x", ajouter comme argument independant_vars = ["nom"])

    # Cette partie devine les paramètres et leur attribue une valeur initiale de 1 pour la recherche
    parametres = modele.make_params()
    for i in parametres :
        modele.set_param_hint(i, value = 1)
    parametres = modele.make_params()

    modele.set_param_hint('tau', value = 1, vary = True, min = 0, max = inf)    # À compléter pour changer les valeurs initiales des paramètres. Le dernier argument est expr (string)

    resultat = modele.fit(y, parametres, x = x)    # Entrer ici le nom de la variable en x et en y, pour le calcul final des paramètres

    # On affiche les valeurs trouvées pour les paramètres :
    valeurs = ""
    for key in resultat.params:
        valeurs += f"{key} = {resultat.params[key].value:.3g} ; incertitude : {resultat.params[key].stderr:.2g}\n"
    return(modele, resultat.params, valeurs, expression)

## On commence par arranger les expressions avec "x" comme variable, car lmfit travaille avec x par défaut :
abscisse = "t"
ordonnee = "uc"
if charge == 0 :
    equation = "E * exp(-1 * t / tau)"
else :
    equation = "E * (1 - exp(-1 * t / tau))"
abscisse1 = abscisse
ordonnee1 = ordonnee
equation1 = equation
equation = re.sub(r"\b"+abscisse+r"\b","x", equation)
abscisse = globals()[abscisse]
ordonnee = globals()[ordonnee]

# On calcule maintenant les grandeurs liées au modèle :
xMod = linspace(min(abscisse), max(abscisse), 30)    # Intervalle de valeurs pour le modèle : (début, fin, nombre de points)
modele, parametres, valeurs, expression = Modele(equation, abscisse, ordonnee)    # Expression du modèle, et variables en abscisse et en ordonnée
expression = f"{ordonnee1} = {equation1}"
yMod = modele.eval(parametres, x = xMod)

# Les paramètres sont définis de façon globale, et utilisables dans des calculs :
for key in parametres:
    globals()[key] = parametres[key].value

Tracer de graphique(s) à partir des tableaux précédents :

In [ ]:
subplot(1, 1, 1)    # lignes, colonnes, numéro du graphique

# Tracé des points expérimentaux :
plot(abscisse, ordonnee, '+', label="")    # abscisse, ordonnée, options : + o . , s points   - -- -. : lignes  b g r y m c k w couleurs ; label est la légende associée à la courbe

# Tracé du modèle :
plot(xMod, yMod, '-', label=f"Modèle :    {expression} \n {valeurs}" )    # trace la courbe du modèle

#axis('equal')    # à décommenter si on veut un repère orthonormé
grid()    # ajoute une grille
legend()    # ajoute la légende au graphique
title("Variation de la tension aux bornes du condensateur")    # mettre le titre désiré entre les guillemets
xlabel("t (s)")    # mettre le nom de l'axe des abscisses entre les guillemets
ylabel("uc (V)");    # mettre le nom de l'axe des ordonnées entre les guillemets. Le point-virgule final évite d'afficher un texte récapitulatif.

Calcul de la valeur de la capacité du condensateur :

In [ ]:
R = 220    #Entrer la valeur de la résistance en Ohm
c = tau / R
print(f"La valeur de la capacité du condensateur est {c:.3} F")